In [1]:

from bs4 import BeautifulSoup as BS
import requests
from time import sleep
import numpy as np
import re
import pandas as pd
import csv

In [ ]:
# Fonction pour récupérer les titres des films et les liens vers leurs pages de critiques des spectateurs
def get_titles_and_review_links(url):
    response = requests.get(url)
    soup = BS(response.text, 'html.parser')
    
    titles_and_links = []
    for item in soup.find_all('a', {'class': 'meta-title-link'}):
        title = item.get_text(strip=True)
        review_link_tag = item.find_next('a', {'class': 'rating-title', 'href': True})
        if review_link_tag:
            review_link = review_link_tag['href']
            titles_and_links.append((title, review_link))
    return titles_and_links

# Fonction pour récupérer les critiques des spectateurs
def get_spectator_reviews(movie_review_url):
    url = f"https://www.allocine.fr{movie_review_url}"  # Lien complet vers les critiques des spectateurs
    response = requests.get(url)
    soup = BS(response.text, 'html.parser')
    
    reviews = []
    for review in soup.find_all('div', {'class': 'content-txt review-card-content'}):
        # Enlever les spoilers
        for spoiler in review.find_all('span', {'class': 'spoiler-content'}):
            spoiler.extract()
        review_text = review.get_text(strip=True)
        reviews.append(review_text)
    return reviews

# URL de base pour les pages des meilleurs films
base_url = 'https://www.allocine.fr/film/meilleurs/?page='

# Récupérer les titres et les liens des 50 meilleurs films (5 pages)
all_titles_and_links = []
for page_number in range(1, 6):  # Pages 1 à 5
    url = f"{base_url}{page_number}"
    titles_and_links = get_titles_and_review_links(url)
    all_titles_and_links.extend(titles_and_links)
    print(f"Page {page_number} : {len(titles_and_links)} films récupérés.")

# Scraper les critiques des spectateurs pour chaque film
movie_reviews = {}
for title, review_link in all_titles_and_links:
    print(f"Récupération des critiques pour le film : {title}")
    reviews = get_spectator_reviews(review_link)
    movie_reviews[title] = reviews

# Afficher les critiques des spectateurs pour chaque film
for title, reviews in movie_reviews.items():
    print(f"\nCritiques des spectateurs pour le film : {title}")
    if reviews:
        for review in reviews:
            print(f"- {review}")
    else:
        print("Aucune critique disponible.")


Page 1 : 10 films récupérés.
Page 2 : 10 films récupérés.
Page 3 : 10 films récupérés.
Page 4 : 10 films récupérés.
Page 5 : 10 films récupérés.
Récupération des critiques pour le film : Forrest Gump
Récupération des critiques pour le film : La Liste de Schindler
Récupération des critiques pour le film : La Ligne verte
Récupération des critiques pour le film : 12 hommes en colère
Récupération des critiques pour le film : Le Parrain
Récupération des critiques pour le film : Les Evadés
Récupération des critiques pour le film : Le Seigneur des anneaux : le retour du roi
Récupération des critiques pour le film : Vol au-dessus d'un nid de coucou
Récupération des critiques pour le film : Le Roi Lion
Récupération des critiques pour le film : The Dark Knight, Le Chevalier Noir
Récupération des critiques pour le film : Il était une fois dans l'Ouest
Récupération des critiques pour le film : Il était une fois en Amérique
Récupération des critiques pour le film : Le Bon, la brute et le truand
Réc

In [9]:
# Exporter les critiques dans un fichier texte
with open("critiques_films.txt", "w", encoding="utf-8") as file:
    for title, reviews in movie_reviews.items():
        file.write(f"Film : {title}\n")
        if reviews:
            for review in reviews:
                file.write(f"{review}\n")
        else:
            file.write("Aucune critique disponible.\n")
        file.write("\n")  # Ligne vide entre les films

print("Les critiques ont été exportées dans le fichier 'critiques_films.txt'.")


Les critiques ont été exportées dans le fichier 'critiques_films.txt'.


In [17]:
movie_titles = [title for title, _ in all_titles_and_links]

# Afficher la liste des titres
print("Liste des titres des films récupérés :")
for title in movie_titles:
    print(title)

Liste des titres des films récupérés :
Forrest Gump
La Liste de Schindler
La Ligne verte
12 hommes en colère
Le Parrain
Les Evadés
Le Seigneur des anneaux : le retour du roi
Vol au-dessus d'un nid de coucou
Le Roi Lion
The Dark Knight, Le Chevalier Noir
Il était une fois dans l'Ouest
Il était une fois en Amérique
Le Bon, la brute et le truand
Interstellar
Gladiator
Pulp Fiction
Le Comte de Monte-Cristo
Le Seigneur des anneaux : la communauté de l'anneau
Django Unchained
Le Seigneur des anneaux : les deux tours
Seven
Gran Torino
Star Wars : Episode V - L'Empire contre-attaque
Le Dictateur
Napoléon vu par Abel Gance partie 1
Les Affranchis
Parasite
Le Parrain, 2e partie
Fight Club
Your Name
Coco
Inception
Retour vers le futur
Green Book : Sur les routes du sud
Dune : Deuxième Partie
Napoléon vu par Abel Gance partie 2
La Vie est belle
Les Temps modernes
Usual Suspects
Le Kid
Le Tombeau des lucioles
Le Silence des agneaux
Whiplash
Psychose
American History X
Star Wars : Episode IV - Un no

In [18]:
import json

# Sauvegarder la liste dans un fichier JSON
with open('movie_titles.json', 'w', encoding='utf-8') as f:
    json.dump(movie_titles, f, ensure_ascii=False, indent=4)